# Minimal version !?

In [ ]:
import sys; sys.path.insert(0,'/Users/ryan/github/prosodic/')
import sys; sys.path.insert(0,'/Users/ryan/github/cadence/')
from cadence.imports import *

In [2]:
novel=read_url(URLS['prideprej'])


In [3]:
def Text(*args,**kwargs):
    return TextModel(*args,**kwargs)

In [16]:
class TextModel(object):
    def __init__(self,txt_or_fn,**kwargs):
        self.fn,self.txt=to_fn_txt(txt_or_fn)
        self.attrs=kwargs

        self._numparas={}
        self._paras_d={}
        self._words_d={}
        self._sylls_d={}
        self._paras_df={}
        self._words_df={}
        self._sylls_df={}
        self._nlp_doc={}
        self._syntax_d={}
        self._syntax_df={}

    ####################################
    ## PARAGRAPHS
    ####################################

    def gen_paras_d(self,**kwargs):
        if not self._paras_d:
            for para_d in tokenize_paras_ld(self.txt, **kwargs):
                self._paras_d[para_d['para_i']] = para_d
            self._numparas=len(self._paras_d)

    def get_para_d(self,para_i,**kwargs):
        return self._paras_d.get(para_i)
    
    def get_paras_ld(self,shuffle_paras=SHUFFLE_PARAS,lim_paras=LIM_PARAS,**kwargs):
        self.gen_paras_d()
        paras_ld=[v for k,v in sorted(self._paras_d.items())]
        if shuffle_paras: random.shuffle(paras_ld)
        if lim_paras: paras_ld=paras_ld[:lim_paras]
        return paras_ld

    def iter_paras_d(self,
            progress=True,
            desc='Iterating over paragraphs',
            **kwargs):
        paras_ld=self.get_paras_ld(**kwargs)
        if progress: paras_ld=tqdm(paras_ld,desc=desc)
        yield from paras_ld
    
    def paras(self,**kwargs):
        return getcache_df(self, '_paras_df', self.iter_paras_d, cache=False, **kwargs)
    


    ####################################
    ## WORDS
    ####################################

    def get_words_ld(self,para_i,**kwargs):
        if not para_i in self._words_d:
            para_d=self.get_para_d(para_i)
            para_str=para_d['para_str']
            self._words_d[para_i]=list(tokenize_sentwords_iter(para_str,para_i=para_i,**kwargs))
        return self._words_d[para_i]
    def iter_words_d(self,**kwargs):
        if not 'desc' in kwargs: kwargs['desc']='Tokenizing sentences and words'
        for para_d in self.iter_paras_d(**kwargs):
            yield from self.get_words_ld(para_d['para_i'])
    def words(self,**kwargs):
        return getcache_df(self, '_words_df', self.iter_words_d, cache=False, **kwargs)
    
    ####################################
    ## SYLLABIFY
    ####################################

    def get_sylls_d(self,word_tok,**kwargs):
        if not word_tok in self._sylls_d:
            self._sylls_d[word_tok]=get_syllable_ld(word_tok,**kwargs)
        yield from self._sylls_d[word_tok]
    
    def iter_sylls_d(self,**kwargs):
        if not 'desc' in kwargs: kwargs['desc']='Tokenizing syllables'
        for word_d in self.iter_words_d(**kwargs):
            sylls_d=self.get_sylls_d(word_d['word_tok'])
            for syll_d in sylls_d:
                yield dict(
                    para_i=word_d['para_i'],
                    sent_i=word_d['sent_i'],
                    word_i=word_d['word_i'],
                    **syll_d
                )
        
    def sylls(self,**kwargs):
        return getcache_df(self, '_sylls_df', self.iter_sylls_d, cache=False, **kwargs)


    ####################################
    ## NLP DOCS
    ####################################

    def get_nlp_doc(self,para_i,**kwargs):
        if not para_i in self._nlp_doc:
            para_words_ld=self.get_words_ld(para_i,**kwargs)
            para_words_ll=tokenize_sentwords_ll(para_words_ld)
            self._nlp_doc[para_i]=get_nlp_doc(para_words_ll,para_i=para_i,**kwargs)
        return self._nlp_doc[para_i]
    def iter_nlp_docs(self,**kwargs):
        if not 'desc' in kwargs: kwargs['desc']='Parsing NLP documents'
        for para_d in self.iter_paras_d(**kwargs):
            yield self.get_nlp_doc(para_d['para_i'],**kwargs)
    def docs(self,**kwargs):
        return list(self.iter_nlp_docs(**kwargs))

    ####################################
    ## NLP FEATS
    ####################################

    def get_syntax_df(self,para_i,**kwargs):
        if not para_i in self._syntax_d:
            para_doc=self.get_nlp_doc(para_i)
            self._syntax_d[para_i]=get_nlp_feats_df(para_doc, **kwargs)
        return self._syntax_d[para_i]
    def iter_syntax_df(self,index=True,**kwargs):
        if not 'desc' in kwargs: kwargs['desc']='Parsing NLP documents'
        for para_d in self.iter_paras_d(**kwargs):
            odf=self.get_syntax_df(para_d['para_i']).assign(para_i=para_d['para_i'])
            if index and type(odf)==pd.DataFrame and len(odf): odf=setindex(odf)
            yield odf
    def syntax(self,**kwargs):
        o=list(self.iter_syntax_df(**kwargs))
        odf=pd.concat(o) if len(o) else pd.DataFrame()
        return odf

In [18]:
joyce_path = os.path.join(PATH_TXTS,'joyce_oxen.txt')
joyce = Text(joyce_path)
joyce.paras()
joyce.words()
joyce.sylls()
# next(joyce.iter_nlp_docs())
joyce.syntax()

Tokenizing syllables: 100%|██████████| 10/10 [00:00<00:00, 129.87it/s]


is_funcword
para_i sent_i word_i word_tok word_ipa_i word_ipa    syll_i syll_str syll_ipa             
2      1      1      hoopsa   1          'hʉːp.sʌ    1      hoo      'hʉːp               0
                                                     2      psa      sʌ                  0
              2      boyaboy  1          'bɔɪ.ʌ.`bɔɪ 1      boy      'bɔɪ                0
                                                     2      a        ʌ                   0
                                                     3      boy      `bɔɪ                0
...                                                                                    ...
62     11     2      my       1          maɪ         1      my       maɪ                 1
              3      own      1          'oʊn        1      own      'oʊn                0
                              2          oʊn         1      own      oʊn                 1
              4      love     1          'lʌv        1      love     'lʌv                0
       12     1      ook      1          'ʉːk        1      ook      'ʉːk                0

[5263 rows x 1 columns]

In [19]:
next(joyce.iter_syntax_df())

Parsing NLP documents:   0%|          | 0/10 [00:00<?, ?it/s]


word_case  ... word_verbform
para_i sent_i word_i word_str   word_lemma word_upos word_xpos word_deprel word_head word_constituency                              word_start_char word_end_char            ...              
17     1      1      350        350        NUM       LS        nummod      7         0-ROOT(1-S(2-NP(3-NNP                          0               3                        ...              
              2      This       this       PRON      DT        nsubj       7         0-ROOT(1-S(2-VP(3-VBP                          4               8                        ...              
              3      meanwhile  meanwhile  ADV       RB        advmod      7         0-ROOT(1-S(2-VP(3-PP(4-IN                      9               18                       ...              
              4      this       this       DET       DT        det         6         0-ROOT(1-S(2-VP(3-PP(4-NP(5-NN                 19              23                       ...              
              5      good       good       ADJ       JJ        amod        6         0-ROOT(1-S(2-VP(3-PP(4-NP(5-NN                 24              28                       ...              
...                                                                                                                                                                     ...  ...           ...
       9      1      Woman's    Woman's    PROPN     NNP       nsubj       2         0-ROOT(1-S(2-S(3-CC                            1420            1427                     ...              
              2      woe        be         VERB      VBP       root        0         0-ROOT(1-S(2-S(3-NP(4-NP(5-NP(6-NNP            1428            1431                     ...           Fin
              3      with       with       ADP       IN        case        5         0-ROOT(1-S(2-S(3-NP(4-NP(5-NP(6-NNP            1432            1436                     ...              
              4      wonder     wonder     NOUN      NN        compound    5         0-ROOT(1-S(2-S(3-NP(4-NP(5-SBAR(6-WHNP(7-WDT   1437            1443                     ...              
              5      pondering. pondering. NOUN      NN        obl         2         0-ROOT(1-S(2-S(3-NP(4-NP(5-SBAR(6-S(7-VP(8-VBD 1444            1454                     ...              

[188 rows x 14 columns]

In [7]:
# next(joyce.iter_nlp_feats())
joyce.feats()

AttributeError: 'TextModel' object has no attribute 'feats'

In [ ]:
display(joyce.paras())
display(joyce.words())
display(joyce.sylls())

,para_i,para_str,para_start_char,para_end_char
0,1,Deshil Holles Eamus. Deshil Holles Eamus. Desh...,0,260
1,2,Hoopsa boyaboy hoopsa! Hoopsa boyaboy hoopsa! ...,262,330
2,3,Universally that person's acumen is esteemed v...,332,2158
3,4,"It is not why therefore we shall wonder if, as...",2160,3406
4,5,347\nTo her nothing already then and thencefor...,3408,4083
...,...,...,...,...
59,60,"You move a motion? Steve boy, you're going it ...",109716,110970
60,61,"405\nGolly, whatten tunket's yon guy in the ma...",110972,112289
61,62,Your attention! We're nae tha fou. The Leith p...,112291,112506
62,63,427\nHark! Shut your obstropolos. Pflaap! Pfla...,112508,112690


,para_i,sent_i,sentpart_i,line_i,word_i,word_pref,word_str,word_tok
0,1,1,1,1,1,,Deshil,deshil
1,1,1,1,1,2,,Holles,holles
2,1,1,1,1,3,,Eamus.,eamus
3,1,2,2,1,1,,Deshil,deshil
4,1,2,2,1,2,,Holles,holles
...,...,...,...,...,...,...,...,...
20303,64,32,51,1,1,,Just,just
20304,64,32,51,1,2,,you,you
20305,64,32,51,1,3,,try,try
20306,64,32,51,1,4,,it,it


,para_i,sent_i,word_i,word_ipa_i,syll_i,word_tok,word_ipa,syll_ipa,syll_str,is_funcword
0,1,1,1,1,1,deshil,'dɛs.hɪl,'dɛs,des,0
1,1,1,1,1,2,deshil,'dɛs.hɪl,hɪl,hil,0
2,1,1,2,1,1,holles,'hɑlz,'hɑlz,holles,0
3,1,1,3,1,1,eamus,'i.mʌs,'i,ea,0
4,1,1,3,1,2,eamus,'i.mʌs,mʌs,mus,0
...,...,...,...,...,...,...,...,...,...,...
33126,64,32,2,1,1,you,juː,juː,you,1
33127,64,32,3,1,1,try,'traɪ,'traɪ,try,0
33128,64,32,4,1,1,it,'ɪt,'ɪt,it,0
33129,64,32,4,2,1,it,ɪt,ɪt,it,1


In [ ]:
novelobj = Text(novel)
display(novelobj.paras())
display(novelobj.words())
display(novelobj.sylls())

gen


Iterating over paragraphs: 100%|██████████| 2233/2233 [00:00<00:00, 6065984.99it/s]


,para_i,para_str,para_start_char,para_end_char
0,1,﻿The Project Gutenberg eBook of Pride and Prej...,0,67
1,2,This eBook is for the use of anyone anywhere i...,69,511
2,3,Title: Pride and Prejudice,513,539
3,4,Author: Jane Austen,541,560
4,5,"Release Date: June, 1998 [eBook #1342]\n[Most ...",562,641
...,...,...,...,...
2228,2229,Section 5. General Information About Project G...,773862,773936
2229,2230,Professor Michael S. Hart was the originator o...,773938,774213
2230,2231,Project Gutenberg-tm eBooks are often created ...,774215,774488
2231,2232,Most people start at our website which has the...,774490,774579


gen


Iterating over paragraphs: 100%|██████████| 2233/2233 [00:00<00:00, 4248.09it/s]


,para_i,sent_i,sentpart_i,line_i,word_i,word_pref,word_str,word_tok
0,1,1,1,1,1,,The,the
1,1,1,1,1,2,,Project,project
2,1,1,1,1,3,,Gutenberg,gutenberg
3,1,1,1,1,4,,eBook,ebook
4,1,1,1,1,5,,of,of
...,...,...,...,...,...,...,...,...
124745,2233,1,4,4,35,,to,to
124746,2233,1,4,4,36,,hear,hear
124747,2233,1,4,4,37,,about,about
124748,2233,1,4,4,38,,new,new


gen


Iterating over paragraphs: 100%|██████████| 2233/2233 [00:01<00:00, 1409.87it/s]


,para_i,sent_i,word_i,word_ipa_i,syll_i,word_tok,word_ipa,syll_ipa,syll_str,is_funcword
0,1,1,1,1,1,the,ðə,ðə,the,1
1,1,1,2,1,1,project,'prɑ.ʤɛkt,'prɑ,pro,0
2,1,1,2,1,2,project,'prɑ.ʤɛkt,ʤɛkt,ject,0
3,1,1,2,2,1,project,prə.'ʤɛkt,prə,pro,0
4,1,1,2,2,2,project,prə.'ʤɛkt,'ʤɛkt,ject,0
...,...,...,...,...,...,...,...,...,...,...
216155,2233,1,37,1,1,about,ə.'baʊt,ə,a,0
216156,2233,1,37,1,2,about,ə.'baʊt,'baʊt,bout,0
216157,2233,1,38,1,1,new,'nuː,'nuː,new,0
216158,2233,1,39,1,1,ebooks,'i.bʊəks,'i,e,0


In [ ]:
joyce.words()

,para_i,sent_i,sentpart_i,line_i,word_i,word_pref,word_str,word_tok
0,1,1,1,1,1,,Deshil,deshil
1,1,1,1,1,2,,Holles,holles
2,1,1,1,1,3,,Eamus.,eamus
3,1,2,2,1,1,,Deshil,deshil
4,1,2,2,1,2,,Holles,holles
...,...,...,...,...,...,...,...,...
20303,64,32,51,1,1,,Just,just
20304,64,32,51,1,2,,you,you
20305,64,32,51,1,3,,try,try
20306,64,32,51,1,4,,it,it


In [ ]:
joyce.sylls()

,para_i,sent_i,word_i,word_ipa_i,syll_i,word_tok,word_ipa,syll_ipa,syll_str,is_funcword
0,1,1,1,1,1,deshil,'dɛs.hɪl,'dɛs,des,0
1,1,1,1,1,2,deshil,'dɛs.hɪl,hɪl,hil,0
2,1,1,2,1,1,holles,'hɑlz,'hɑlz,holles,0
3,1,1,3,1,1,eamus,'i.mʌs,'i,ea,0
4,1,1,3,1,2,eamus,'i.mʌs,mʌs,mus,0
...,...,...,...,...,...,...,...,...,...,...
33126,64,32,2,1,1,you,juː,juː,you,1
33127,64,32,3,1,1,try,'traɪ,'traɪ,try,0
33128,64,32,4,1,1,it,'ɪt,'ɪt,it,0
33129,64,32,4,2,1,it,ɪt,ɪt,it,1


In [ ]:
joyce.words()

Iterating over paragraphs: 100%|██████████| 64/64 [00:00<00:00, 79301.46it/s]


,para_i,sent_i,sentpart_i,line_i,word_i,word_pref,word_str,word_tok
0,1,1,1,1,1,,Deshil,deshil
1,1,1,1,1,2,,Holles,holles
2,1,1,1,1,3,,Eamus.,eamus
3,1,2,2,1,1,,Deshil,deshil
4,1,2,2,1,2,,Holles,holles
...,...,...,...,...,...,...,...,...
20303,64,32,51,1,1,,Just,just
20304,64,32,51,1,2,,you,you
20305,64,32,51,1,3,,try,try
20306,64,32,51,1,4,,it,it


In [ ]:
joyce.sentwords()

AttributeError: 'Text' object has no attribute 'sentwords'

In [ ]:
next(joyce.iter_sentwords_d(shuffle_paras=True))

{'para_i': 41,
 'sent_i': 1,
 'sentpart_i': 1,
 'line_i': 1,
 'word_i': 1,
 'word_pref': '',
 'word_str': 'But',
 'word_tok': 'but'}

In [ ]:
joyce.sents()

,para_i,sent_i,sent_str,sent_start_char,sent_end_char
0,1,1,Deshil Holles Eamus.,0,20
1,1,2,Deshil Holles Eamus.,21,41
2,1,3,Deshil Holles Eamus.,42,62
3,1,4,"Send us bright one, light one, Horhorn, quicke...",63,128
4,1,5,"Send us bright one, light one, Horhorn, quicke...",129,194
...,...,...,...,...,...
1108,64,28,"You'll need to rise precious early, you sinner...",1164,1257
1109,64,29,Pflaaaap!,1258,1267
1110,64,30,Not half.,1268,1277
1111,64,31,He's got a coughmixture with a punch in it for...,1278,1359


In [ ]:
joyce.words()

NameError: name 'sep_line' is not defined

In [ ]:
# dfwords=pd.DataFrame(joyce.iter_words_d())
# dfwords

In [ ]:
joyce.words()

,para_i,sent_i,word_i,word_str,word_tok
0,1,1,0,Deshil,deshil
1,1,1,1,Holles,holles
2,1,1,2,Eamus.,eamus
3,1,2,0,Deshil,deshil
4,1,2,1,Holles,holles
...,...,...,...,...,...
20303,64,32,0,Just,just
20304,64,32,1,you,you
20305,64,32,2,try,try
20306,64,32,3,it,it


In [ ]:
next(joyce.iter_sents())

AttributeError: 'Text' object has no attribute 'iter_sents'

In [ ]:
for x in joyce.iparas(lim=2,shuffle=True):
    print(x)

{'para_i': 63, 'para_str': '427\nHark! Shut your obstropolos. Pflaap! Pflaap! Blaze on. There she goes. Brigade! Bout ship. Mount street way. Cut up! Pflaap! Tally ho. You not come? Run, skelter, race. Pflaaaap!', 'para_start_char': 112508, 'para_end_char': 112690}
{'para_i': 8, 'para_str': "Of that house A. Horne is lord. Seventy beds keeps he there teeming mothers are wont that they lie for to thole and bring forth bairns hale so God's angel to Mary quoth. Watchers twey there walk, white sisters in ward sleepless. Smarts they still, sickness soothing: in twelve moons thrice an hundred. Truest bedthanes they twain are, for Horne holding wariest ward.", 'para_start_char': 5033, 'para_end_char': 5399}


In [ ]:
joyce.paras()

,para_i,para_str,para_start_char,para_end_char
0,1,Deshil Holles Eamus. Deshil Holles Eamus. Desh...,0,260
1,2,Hoopsa boyaboy hoopsa! Hoopsa boyaboy hoopsa! ...,262,330
2,3,Universally that person's acumen is esteemed v...,332,2158
3,4,"It is not why therefore we shall wonder if, as...",2160,3406
4,5,347\nTo her nothing already then and thencefor...,3408,4083
...,...,...,...,...
59,60,"You move a motion? Steve boy, you're going it ...",109716,110970
60,61,"405\nGolly, whatten tunket's yon guy in the ma...",110972,112289
61,62,Your attention! We're nae tha fou. The Leith p...,112291,112506
62,63,427\nHark! Shut your obstropolos. Pflaap! Pfla...,112508,112690


In [ ]:
joy

NameError: name 'joy' is not defined

In [ ]:
joyce.paras()

[{'para_i': 1,
  'para_str': "Valuing himself not a little upon his elegance, being indeed a proper man of his person, this talkative now applied himself to his dress with animadversions of some heat upon the sudden whimsy of the atmospherics while the company lavished their encomiums upon the project he had advanced. The young gentleman, his friend, overjoyed as he was at a passage that had befallen him, could not forbear to tell it his nearest neighbour. Mr Mulligan, now perceiving the table, asked for whom were those loaves and fishes and, seeing the stranger, he made him a civil bow and said, Pray, sir, was you in need of any professional assistance we could give? Who, upon his offer, thanked him very heartily, though preserving his proper distance, and replied that he was come there about a lady, now an inmate of Horne's house, that was in an interesting condition, poor lady, from woman's woe (and here he fetched a deep sigh) to know if her happiness had yet taken place. Mr Dixon,

In [ ]:
t.paras()

[{'para_i': 1,
  'para_str': 'In Xanadu did Kubla Khan\nA stately pleasure-dome decree:\nWhere Alph, the sacred river, ran\nThrough caverns measureless to man\n   Down to a sunless sea.',
  'para_start_char': 0,
  'para_end_char': 151}]

In [ ]:







    # ## sents
    # def iter_sents_d(self,
    #         lim_sents=None,
    #         **kwargs):

    #     for para_d in self.iter_paras_d(**kwargs):
    #         para_i=para_d['para_i']
    #         para_str=para_d['para_str']
    #         key=(para_i,)

    #         if key in self._sents:
    #             yield from self._sents[key]
    #         else:
    #             self._sents[key]=ld=[]
    #             sent_toks=tokenize_sents_txt(para_str)
    #             offset_start=-1
    #             for sent_i,sent_str in enumerate(sent_toks):
    #                 #para_left=para_str[lastmax:]
    #                 offset_start=para_str.find(sent_str,offset_start+1)
    #                 offset_end=offset_start+len(sent_str)
    #                 sent_d=dict(
    #                     para_i=para_i,
    #                     sent_i=sent_i+1,
    #                     # para_str=para_str,
    #                     sent_str=sent_str,
    #                     #sent_str2=para_str[offset_start : offset_end],
    #                     sent_start_char=offset_start,
    #                     sent_end_char=offset_end,
    #                 )
    #                 ld.append(sent_d)
    #                 yield sent_d
                                

    # ## words
    # def iter_words_d(self,**kwargs):
    #     for sent_d in self.iter_sents_d(**kwargs):
    #         para_i,sent_i,sent_str=sent_d['para_i'],sent_d['sent_i'],sent_d['sent_str']
    #         key=(para_i,sent_i)
    #         if key in self._words:
    #             yield from self._words[key]
    #         else:
    #             self._words[key]=ld=[]
    #             tokiter=tokenize_words_txt(sent_str,**kwargs)
    #             for word_i,realtok in enumerate(tokiter):
    #                 prefstr,wordstr,sufstr=split_punct(realtok)
    #                 tokstr=wordstr+sufstr
    #                 if sep_line in prefstr and realtok.strip(): line_i+=1
                    
    #                 odx_word=dict(
    #                     para_i=para_i,
    #                     sent_i=sent_i+1,
    #                     sentpart_i=linepart_i,
    #                     line_i=line_i,
    #                     word_i=tok_i+1,
    #                     word_pref=prefstr,
    #                     word_str=tokstr,
    #                     word_tok=wordstr.lower(),
    #                 )
    #                 ld.append(od)
    #                 yield odx_word
    #                 if set(realtok)&set(seps_phrase): linepart_i+=1


    # def paras(self,**y): return pd.DataFrame(self.iter_paras_d(**y))
    # def sents(self,**y): return pd.DataFrame(self.iter_sents_d(**y))
    # def words(self,**y): return pd.DataFrame(self.iter_words_d(**y))



# def do_Text_iter_sents(para_d):
#     pass